### Import the project from BigML

In [2]:
#BigML project
from pandas import read_csv
import bigml.api
from bigml.api import BigML
#api= BigML (project='project/5d94a52e5a213962e2000331')
api=BigML()

### 1st Model (ensemble) on train splitted dataset

In [4]:
#Upload the modified train set
source = api.create_source('fulltrain_dataframe.csv')

In [5]:
api.ok(source)

True

In [6]:
#Create the dataset
dataset_1 = api.create_dataset(source)

In [7]:
api.ok(dataset_1)

True

In [8]:
#Split the trainfull in train80/val20
train_dataset = api.create_dataset(
    dataset_1, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_1, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [9]:
api.ok(train_dataset)

True

In [10]:
api.ok(val)

True

In [11]:
api.download_dataset(train_dataset, filename='train_dataset.csv')

'train_dataset.csv'

In [12]:
api.download_dataset(val, filename='val.csv')

'val.csv'

In [13]:
#Launch the model ensemble
model = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [14]:
api.ok(model)

True

In [15]:
#Launch the batch prediction on the validation set
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True, "confidence": True, "probabilities": True})

In [16]:
api.ok(batch_prediction)

True

In [17]:
#Download the prediction 
api.download_batch_prediction(batch_prediction,
    filename='../prediction_credit.csv')

'../prediction_credit.csv'

### Apply the Model on the test set

In [22]:
#Upload the modified test
test_source = api.create_source('test_dataframe.csv')

In [23]:
api.ok(test_source)

True

In [24]:
#Create the dataset test
dataset_2 = api.create_dataset(test_source)

In [25]:
api.ok(dataset_2)

True

In [26]:
#Model ensemble on the test set
model = api.create_ensemble(dataset_1, {"objective_field":"SeriousDlqin2yrs"})

In [27]:
api.ok(model)

True

In [28]:
#Batch prediction on the test set + add column probabilities
batch_prediction = api.create_batch_prediction(model, dataset_2, {"all_fields": True, "confidence": True, "probabilities": True})

In [29]:
api.ok(batch_prediction)

True

In [30]:
#Evaluation on the test
evaluation = api.create_evaluation(model, dataset_1)
api.ok(evaluation)

True

In [31]:
 api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])

0.88162


In [32]:
api.download_batch_prediction(batch_prediction,
    filename='prediction_credit_final.csv')

'../Credit_p/prediction_credit_final.csv'

In [33]:
# Import the new prediction and remove the columns/replace Id and Probability
import pandas as pd
df = pd.read_csv('prediction_credit_final.csv')
df_kgle = df [['field1', '1 probability']] 

In [34]:
col_names_subm = df_kgle.columns.values
col_names_subm[1] = 'Probability'
col_names_subm[0] = 'Id'

df_kgle.columns = col_names_subm
df_kgle.head(5)

,Id,Probability
0,1,0.04978
1,2,0.03378
2,3,0.08006
3,4,0.07442
4,5,0.06891


In [35]:
df_kgle.to_csv('prediction_credit_final_clean.csv', index=False)

In [36]:
import kaggle
kaggle.api.competition_submit("prediction_credit_final_clean.csv", "test", "GiveMeSomeCredit")

100%|██████████| 1.66M/1.66M [00:05<00:00, 328kB/s] 


Successfully submitted to Give Me Some Credit

In [1]:
score = 0.85753
print("Kaggle score :", score)

Kaggle score : 0.85753
